# Tutorial for sipmod

Run the following cell to download libraries:

In [ ]:
!git clone https://github.com/yuccadrew/sip.git

# Import the finite element mesh

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('sip'))

import numpy as np
import scipy.sparse.linalg as spl

from sipmod import *

# Import mesh and compute basis functions
mesh_prefix = 'sip/docs/examples/meshes/mesh_ex01'
mesh = MeshTri.read(mesh_prefix+'.1', scale_factor=1e-6)
mesh.draw().show()

# Compute the basis functions

In [ ]:
basis = CellBasisTri(mesh)
basis

# Define the parameters

In [ ]:
# Define function of nonuniform load
@Form
def domain_params(x, y, *args, **kwargs):
    r = np.sqrt(x**2 + y**2)
    f = np.zeros(len(r))
    mask = r > 0
    f[mask] = np.pi / 2 * (1 / r[mask] * np.sin(np.pi * r[mask] / 2) +
                           np.pi / 2 * np.cos(np.pi * r[mask] / 2))
    f[~mask] = np.pi**2 / 2
    return {'f': f}

# Define function of nonhomogeneous boundary condition
@Form
def boundary_params(x, y, *args, **kwargs):
    r = np.sqrt(x**2 + y**2)
    s = np.cos(np.pi * r / 2)
    return {'s': s}

# Define the forms

In [ ]:
# Define bilinear form for assembling LSH of the linear system
@BilinearForm(c=1)
def laplace(u, v, w):
    return (
        w.c * u.grad[0] * v.grad[0] +
        w.c * u.grad[1] * v.grad[1]
    )

# Define linear form for assembling RHS of the linear system
@LinearForm(w=domain_params)
def nonuniform_load(_, w):
    return w.f / 3

# Assemble and solve the linear system

In [ ]:
# Assemble the system of equations
A = laplace.assemble(basis).todefault()
b = nonuniform_load.assemble(basis).todefault()

# Incoorprate the Dirichlet boundary condition
boundary_nodes = mesh.boundary_nodes('outer')
D = boundary_nodes['id']
s = boundary_params(**boundary_nodes)['s']
A, b = set_first_kind_bc(A, b, s, D)

# Call the sparse linear system solver
x = spl.spsolve(A, b)

# Visualize the solution

In [ ]:
mesh.plot(x, cmap='jet').show()